# 3. Tensorflow estimators

Estimator are tensorflow's high-level api for creating neural network models.
A estimator provides easy interface for training, evaluating and predicting 
with neural networks.

A main reason to use estimator is to avoid writing your own code for training
and focus on the network structure. 

A estimator can be easily trained on different resources (CPU, GPU, Cloud, etc）

## 3.1 Define an Estimator - dataset

The first step to use estimators is to translate the data to something
it could understand.

Tensorflow dataset can be easily created from numpy arrays:

```python
tf.data.Dataset.from_tensor_slices({"x": x_train, "y": y_train})
```

However, estimator expect the dataset to be a function that returns 
a dataset, instead of a dataset itself.

The following code creates datasets of {x,y} values.
In addition, the dataset is shuffled and batched to 5-element batches.  
The use of mini-batches like this is common in the training of neural networks，so that the gradient can be updated by calculating only part of the dataset.


In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [ ]:
import numpy as np
import tensorflow as tf

x_train = np.linspace(0, 10,1000)[:,np.newaxis]
y_train = np.sin(x_train*2)
def dataset():
    return tf.data.Dataset.from_tensor_slices(
        {"x": x_train, "y": y_train}).repeat().shuffle(1000).batch(500)

The following code shows a minibatch of the dataset:

In [ ]:
iterator = dataset().make_one_shot_iterator()
next_element = iterator.get_next()
sess = tf.Session()
sess.run(next_element)
sess.close()

## Define an Estimator - model function

The estimator is defined by a model function.  


The input parameters of the model function is fixed.  
The  model function should return corresponding `EstimatorSpec` according to the `mode` parameter.

In this simple example, the model function uses two densely connected layers of neurons and outputs a single value.
The model function returns the prediction when `mode` is `PREDICT`, and returns a AdamOptimizer when `mode` is `TRAIN`.  

(Adam is a somehow improved version of gradient descent, again, we will not focus on the details about the optimizer)



In [ ]:
def model_fn(features, labels, mode, params):
    x = features['x']
    
    # This defines a neural network with three hidden layers
    hidden1 = tf.layers.dense(x,       50, activation='tanh')
    hidden2 = tf.layers.dense(hidden1, 50, activation='tanh')
    hidden3 = tf.layers.dense(hidden2, 50, activation='tanh')
    # The output node do not have a activation function,
    # so that the output value is not constrained
    y_pred = tf.layers.dense(hidden3, 1)

    if mode == tf.estimator.ModeKeys.TRAIN:
        y = features['y']
        loss = tf.losses.mean_squared_error(y, y_pred)
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)
        
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = y_pred
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

## Simple neural network with Estimator

The estimator can then be created with the model function.

In [ ]:
# Here the model_dir specifies where the training data is saved
# Change the directory when you try out different parameters
estimator = tf.estimator.Estimator(model_dir="simple_estimator", model_fn=model_fn) 
estimator.train(input_fn=dataset, max_steps=5000)

Using the estimator is to predict somehow not straight forward.
It requires you to provide again a dataset for prediction.
Use the following code to see your fitting results.

You can also open a tensorboard to see how how the learning curve goes.
Adjust your parameters accordingly to see how things change.  
To use tensorboard, go back to the directory page, and click New->Tensorboad.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

x_pred = np.linspace(0,10,1000)[:,np.newaxis]

# Use the estimator to predict
pred_ds = lambda: tf.data.Dataset.from_tensor_slices({'x':x_pred}).batch(1000)
y_pred = estimator.predict(pred_ds, yield_single_examples=False)
y_pred = next(y_pred)

plt.plot(x_pred, y_pred, label='Prediction')
plt.plot(x_train, y_train, label='Training')
plt.legend()

The above example is just a simplest demonstration of estimator.

If you are after some challenge, you might want to implement some 
real-world applications of neural networks.

See here for a collection of tutorial: https://github.com/GoogleCloudPlatform/tf-estimator-tutorials

In [ ]:
# Or get it with the following block
!git clone https://github.com/GoogleCloudPlatform/tf-estimator-tutorials.git